In [ ]:
import pyomo.environ as pyo
from pyomo.opt.results import SolverStatus
import pandas as pd
import json
import math
import statistics
import random
import re

In [ ]:
class Player:
    pref_score = 0
    team = None
    board = None
    req_met = False

    def __init__(self, name, rating, friends, avoid, date, alt, previous_season_alt):
        self.name = name
        self.rating = rating
        self.friends = friends
        self.avoid = avoid
        self.date = date
        self.alt = alt
        self.previous_season_alt = previous_season_alt

    @classmethod
    def player_from_json(cls, player):
        return cls(
            player['name'],
            player['rating'],
            player['friends'],
            player['avoid'],
            player['date_created'],
            player['prefers_alt'],
            player.get('previous_season_alternate', False)
        )

    def __repr__(self):
        return str((self.name, self.board, self.rating, self.req_met))

    def __lt__(self, other):
        return True

    def set_pref_score(self):
        self.pref_score = 0
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.pref_score += 1
            else:
                self.pref_score -= 1
        for avoid in self.avoid:
            if avoid in self.team.get_boards():
                self.pref_score -= 1
        # player with more than 5 choices can be <5 preference even if all teammates are preferred

    def set_req_met(self):
        self.req_met = False
        if not self.friends:
            self.req_met = None
        for friend in self.friends:
            if friend in self.team.get_boards():
                self.req_met = True


class Team:
    def __init__(self, boards):
        self.boards = [None for x in range(boards)]

    def __str__(self):
        return str((self.boards, self.team_pref_score, self.get_mean()))

    def __repr__(self):
        return "Team:{0}".format(id(self))

    def __lt__(self, other):
        return True

    def changeBoard(self, board, new_player):
        # updates the player on a board and updates that player's team attribute
        if self.boards[board]:
            self.boards[board].team = None
        self.boards[board] = new_player
        if new_player.team:
            new_player.team.boards[board] = None
        new_player.team = self

    def get_mean(self, expected_rating=False):
        # expected_rating is an unused parameter in this version.
        # it is used by the tournament.models.Team.get_mean method.
        ratings = [board.rating for board in self.boards]
        mean = sum(ratings) / len(ratings)
        return mean

    def get_boards(self):
        return self.boards

    def get_player(self, board):
        return self.boards[board]

    def set_team_pref_score(self):
        self.team_pref_score = sum([x.pref_score for x in self.boards])
        
def split_into_equal_groups_by_rating(players, group_number):
    players.sort(key=lambda player: player.rating, reverse=True)
    avg = len(players) / float(group_number)
    players_split = []
    last = 0.0
    while round(last) < len(players):
        players_split.append(players[int(round(last)):int(round(last + avg))])
        last += avg
    return players_split


def get_rating_bounds_of_split(split):
    min_ratings = [min([p.rating for p in board]) for board in split]
    max_ratings = [max([p.rating for p in board]) for board in split]
    min_ratings[-1] = 0
    max_ratings[0] = 5000
    return list(zip(min_ratings, max_ratings))

def flatten(lst):
    return [item for sub_lst in lst for item in sub_lst]

In [ ]:
# Load data
with open("data.json") as f:
    playerdata = json.load(f)
    
# Parameters
boards = 10
rounds = 8
balance = 0.85
balance_type = 'none' # rating, winexp, or none

In [ ]:
players = []
for player in playerdata:
    if player['has_20_games'] and player['in_slack']:
        players.append(Player.player_from_json(player))
    else:
        pass
        # print("{0} skipped".format(player['name']))
players.sort(key=lambda player: player.rating, reverse=True)

# Split into those that want to be alternates vs those that do not.
alternates = [p for p in players if p.alt]
players = [p for p in players if not p.alt]

# splits list of Player objects into near equal lists, sectioned by rating
players_split = split_into_equal_groups_by_rating(players, boards)
team_rating_bounds = get_rating_bounds_of_split(players_split)

num_teams = int(math.ceil((len(players_split[0]) * balance) / 2.0) * 2)

# separate latest joining players into alternate lists as required
for n, board in enumerate(players_split):
    board.sort(key=lambda player: (0 if player.previous_season_alt else 1, player.date))
    alternates.extend(board[num_teams:])
    del board[num_teams:]
    board.sort(key=lambda player: player.rating, reverse=True)

alts_split = split_into_equal_groups_by_rating(alternates, boards)
alt_rating_bounds = get_rating_bounds_of_split(alts_split)

players = flatten(players_split)

for n, board in enumerate(players_split):
    for player in board:
        player.board = n

def convert_name_list(string_of_names, players):
    pattern = r"([^-_a-zA-Z0-9]|^){0}([^-_a-zA-Z0-9]|$)"
    return [player for player in players
            if re.search(pattern.format(player.name), string_of_names, flags=re.I)]

for player in players:
    filtered_players = [p for p in players if p.board != player.board]
    player.friends = convert_name_list(player.friends, filtered_players)
    player.avoid = convert_name_list(player.avoid, filtered_players)

In [ ]:
def update_pref(players, teams):  # update preference scores
    for player in players:
        player.set_pref_score()
    for team in teams:
        team.set_team_pref_score()


def update_sort(players, teams):  # based on preference score high to low
    players.sort(key=lambda player: (player.team.team_pref_score, player.pref_score), reverse=False)
    teams.sort(key=lambda team: team.team_pref_score, reverse=False)

In [ ]:
# randomly shuffle players
for board in players_split:
    random.shuffle(board)

teams = []
for n in range(num_teams):
    teams.append(Team(boards))
for n, board in enumerate(players_split):
    for team, player in enumerate(board):
        teams[team].changeBoard(n, player)

In [ ]:
# Calculated parameters
target_team_rating = statistics.mean([player.rating for player in players])
target_team_winexp = rounds/2

In [ ]:
def build_team_optimization_model(players, boards, 
                                  num_teams, target_team_rating, target_team_winexp,
                                  friend_cost=1, avoid_cost=-5,
                                  rating_viol_cost=-1, rating_threshold=10, 
                                  winexp_viol_cost=-50, winexp_threshold=0.05):
    """
    Returns a pyomo model, which can be solved to generate teams 
    (see https://pyomo.readthedocs.io/en/stable/solving_pyomo_models.html).

            Parameters:
                    players (list of objects): List of Player class instances to be placed into teams.
                    boards (int): Number of boards on each team.
                    num_teams (int): Number of teams.
                    target_team_rating (float): The average rating of the player pool.
                    target_team_winexp (float): The number of matches a team should be expected to win.
                    friend_cost (float): The 'happiness' value assigned for one friend pairing on the same team. 
                        Should be a positive number, but this is not enforced.
                    avoid_cost (float): The 'unhappiness' value assigned for a player being placed on the same
                        team as a player they avoided. Should be a negative number, but this is not enforced.
                    rating_viol_cost (float): The 'unhappiness' value assigned for violating the rating
                        threshold by 1 point. Should be a negative number, but this is not enforced.
                    rating_threshold (float): The one-sided size of the window for acceptable team average rating.
                    winexp_viol_cost (float): The 'unhappiness' value assigned for violating the full season 
                        team win expectation by 1 point. Should be a negative number, but this is not enforced.
                    winexp_threshold (float): The one-sided size of the window for acceptable full season team 
                        win expectation.

            Returns:
                    model (object): Instance of pyomo.environ.AbstractModel
    """
    
    # Model
    model = pyo.AbstractModel()

    # Sets
    model.players = pyo.Set(initialize=[player.name for player in players])
    model.boards = pyo.Set(initialize=[board_num for board_num in range(boards)])
    model.teams = pyo.Set(initialize=[team_num for team_num in range(num_teams)])
    model.avoids = pyo.Set(initialize=[(player.name, avoided_player.name) for player in players 
                                       for avoided_player in player.avoid], within=model.players*model.players, dimen=2)
    model.friends = pyo.Set(initialize=[(player.name, friended_player.name) for player in players 
                                        for friended_player in player.friends], within=model.players*model.players, dimen=2)

    # Parameters
    model.param_player_rating = pyo.Param(model.players, 
                                          initialize={player.name:player.rating for player in players}, 
                                          within=pyo.Reals)
    model.param_player_expscore = pyo.Param(model.players, 
                                            initialize={player.name:4.0 for player in players}, 
                                            within=pyo.NonNegativeReals)
    model.param_player_boards = pyo.Param(model.players, 
                                          initialize={player.name:player.board for player in players}, 
                                          within=pyo.NonNegativeIntegers)
    model.param_friend_cost = pyo.Param(initialize=friend_cost, within=pyo.Reals)
    model.param_avoid_cost = pyo.Param(initialize=avoid_cost, within=pyo.Reals)
    model.param_rating_viol_cost = pyo.Param(initialize=rating_viol_cost, within=pyo.Reals)
    model.param_winexp_viol_cost = pyo.Param(initialize=winexp_viol_cost, within=pyo.Reals)
    model.param_rating_threshold = pyo.Param(initialize=rating_threshold, within=pyo.Reals)
    model.param_winexp_threshold = pyo.Param(initialize=winexp_threshold, within=pyo.Reals)

    # Decision variables
    model.dvar_x = pyo.Var(model.players, model.boards, model.teams, within=pyo.Binary, initialize=0)

    # Soft constraint penalty decision variables
    model.pen_friends = pyo.Var(model.friends, model.teams, within=pyo.Binary, initialize=0)
    model.pen_avoids = pyo.Var(model.avoids, model.teams, within=pyo.Binary, initialize=0)
    model.pen_balance_rating = pyo.Var(model.teams, within=pyo.NonNegativeReals, initialize=0)
    model.pen_balance_winexp = pyo.Var(model.teams, within=pyo.NonNegativeReals, initialize=0)

    # Objective
    def objective_function(model):
        return model.param_friend_cost*pyo.summation(model.pen_friends) \
        + model.param_avoid_cost*pyo.summation(model.pen_avoids) \
        + model.param_rating_viol_cost*pyo.summation(model.pen_balance_rating) \
        + model.param_winexp_viol_cost*pyo.summation(model.pen_balance_winexp)
    model.obj = pyo.Objective(rule=objective_function, sense=pyo.maximize)

    # Hard Constraint: Each player must be on one and only one team
    def constraint_one_team(model, p):
        return sum(model.dvar_x[p, b, t] for b in model.boards for t in model.teams) == 1
    model.cons_one_team = pyo.Constraint(model.players, rule=constraint_one_team)
    
    # Hard Constraint: Each team must have exactly the right number of players
    def constraint_right_number_of_players(model, t):
        return sum(model.dvar_x[p, b, t] for p in model.players for b in model.boards) == boards
    model.cons_right_number_of_players = pyo.Constraint(model.teams, rule=constraint_right_number_of_players)

    # Hard Constraint: Each player must be on their own board
    def constraint_own_board(model, p):
        return sum(model.dvar_x[p, model.param_player_boards[p], t] for t in model.teams) == 1
    model.cons_own_board = pyo.Constraint(model.players, rule=constraint_own_board)
    
    # Hard Constraint: One player on each board for each team
    def constraint_one_of_each_board(model, b, t):
        return sum(model.dvar_x[p, b, t] for p in model.players) == 1
    model.cons_one_of_each_board = pyo.Constraint(model.boards, model.teams, rule=constraint_one_of_each_board)

    # Soft Constraint: Teams must be balanced, using rating
    def constraint_team_balance_rating_lb(model, t):
        return sum(model.param_player_rating[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        >= boards*target_team_rating - model.param_rating_threshold - model.pen_balance_rating[t]
    model.cons_team_balance_rating_lb = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_lb)

    def constraint_team_balance_rating_ub(model, t):
        return sum(model.param_player_rating[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        <= boards*target_team_rating + model.param_rating_threshold + model.pen_balance_rating[t]
    model.cons_team_balance_rating_ub = pyo.Constraint(model.teams, rule=constraint_team_balance_rating_ub)

    # Soft Constraint: Teams must be balanced, using expected wins
    def constraint_winexp_rating_lb(model, t):
        return sum(model.param_player_expscore[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        >= target_team_winexp - model.param_winexp_threshold - model.pen_balance_winexp[t]
    model.cons_winexp_rating_lb = pyo.Constraint(model.teams, rule=constraint_winexp_rating_lb)

    def constraint_winexp_rating_ub(model, t):
        return sum(model.param_player_expscore[p]*model.dvar_x[p, b, t] for p in model.players for b in model.boards) \
        <= target_team_winexp + model.param_winexp_threshold + model.pen_balance_winexp[t]
    model.cons_winexp_rating_ub = pyo.Constraint(model.teams, rule=constraint_winexp_rating_ub)

    # Soft Constraint: Players should not be placed on the same team as a player on their avoid list
    def constraint_avoid_list(model, p1, p2, t):
        return 2*model.pen_avoids[(p1, p2), t] \
        <= sum(model.dvar_x[p1, b, t]+model.dvar_x[p2, b, t] for b in model.boards)
    model.cons_avoid_list = pyo.Constraint(model.avoids, model.teams, rule=constraint_avoid_list)

    # Soft Constraint: Players should be placed with their friends
    def constraint_friend_list(model, p1, p2, t):
        return 2*model.pen_friends[(p1, p2), t] \
        <= sum(model.dvar_x[p1, b, t] + model.dvar_x[p2, b, t] for b in model.boards)
    model.cons_friend_list = pyo.Constraint(model.friends, model.teams, rule=constraint_friend_list)
    
    return model

In [ ]:
model = build_team_optimization_model(players, boards, num_teams, target_team_rating, target_team_winexp)
instance = model.create_instance()

# Simple model - deactivate all team balance constraints
if balance_type == 'none':
    instance.cons_team_balance_rating_lb.deactivate()
    instance.cons_team_balance_rating_ub.deactivate()
    instance.cons_winexp_rating_lb.deactivate()
    instance.cons_winexp_rating_ub.deactivate()
elif balance_type == 'rating':
    instance.cons_winexp_rating_lb.deactivate()
    instance.cons_winexp_rating_ub.deactivate()
elif balance_type == 'winexp':
    instance.cons_team_balance_rating_lb.deactivate()
    instance.cons_team_balance_rating_ub.deactivate()

In [ ]:
opt = pyo.SolverFactory('glpk')
opt.options['tmlim'] = 600
result = opt.solve(instance, tee=True)

In [ ]:
result.Solver.Status = SolverStatus.warning
instance.solutions.load_from(result)

In [ ]:
result

In [ ]:
instance.obj.display()

In [ ]:
team_assignments = []
for v in instance.component_objects(pyo.Var, active=True):
    print(v.name)
    if v.name == 'dvar_x':
        for index in v:
            if pyo.value(v[index]) > 0:
                team_assignments.append(index)
    for index in v:
        if pyo.value(v[index]) > 0:
            print ("   ",index, pyo.value(v[index]))
a = pd.DataFrame(team_assignments, columns=['name', 'board', 'team'])

In [ ]:
a = a.merge(pd.DataFrame(playerdata)[['name', 'rating']])
a[['team', 'rating']].groupby('team', as_index=False).mean()